## Image Classification
Deze versie is gemaakt op 11-12 na het maken van de grotere set eigen afbeeldingen.

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import onnx
import onnxruntime as ort
import numpy as np

In [3]:
# Define hyperparameters
batch_size = 32
num_epochs =50
learning_rate = 0.00002

In [3]:
# Data transformations 
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Lambda(lambda x: x / 255.0)  # Normalize by dividing by 255
])

In [4]:
# Load datasets CROPPED NO PADDING
train_dataset = datasets.ImageFolder('data/cropped_no_padding/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder('data/cropped_no_padding/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# Load datasets CROPPED NO PADDING
train_dataset = datasets.ImageFolder('data/cropped_no_padding/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder('data/cropped_no_padding/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Calculate the number of outputs for the first fully connected layer
final_feature_map_size = 224 // (2 * 2)  # After two 2x2 max pooling layers

In [6]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(32 * final_feature_map_size * final_feature_map_size, 100)
        self.fc2 = nn.Linear(100, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # Flatten
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [7]:
# Instantiate the model, define the loss function and the optimizer
num_classes = len(train_dataset.classes)
model = CNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Training loop
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/50], Loss: 0.1173
Epoch [2/50], Loss: 0.1141
Epoch [3/50], Loss: 0.0442
Epoch [4/50], Loss: 0.0389
Epoch [5/50], Loss: 0.0165
Epoch [6/50], Loss: 0.0228
Epoch [7/50], Loss: 0.0055
Epoch [8/50], Loss: 0.0102
Epoch [9/50], Loss: 0.0255
Epoch [10/50], Loss: 0.0104
Epoch [11/50], Loss: 0.0103
Epoch [12/50], Loss: 0.0054
Epoch [13/50], Loss: 0.0238
Epoch [14/50], Loss: 0.0149
Epoch [15/50], Loss: 0.0045
Epoch [16/50], Loss: 0.0076
Epoch [17/50], Loss: 0.0023
Epoch [18/50], Loss: 0.0036
Epoch [19/50], Loss: 0.0019
Epoch [20/50], Loss: 0.0019
Epoch [21/50], Loss: 0.0019
Epoch [22/50], Loss: 0.0004
Epoch [23/50], Loss: 0.0010
Epoch [24/50], Loss: 0.0020
Epoch [25/50], Loss: 0.0161
Epoch [26/50], Loss: 0.0014
Epoch [27/50], Loss: 0.0021
Epoch [28/50], Loss: 0.0017
Epoch [29/50], Loss: 0.0021
Epoch [30/50], Loss: 0.0009
Epoch [31/50], Loss: 0.0006
Epoch [32/50], Loss: 0.0006
Epoch [33/50], Loss: 0.0005
Epoch [34/50], Loss: 0.0005
Epoch [35/50], Loss: 0.0005
Epoch [36/50], Loss: 0.0010
E

In [9]:
# Testing loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Accuracy of the model on the test images: 99.55%


In [10]:
# Aantal klassen in je model (bijvoorbeeld 75 klassen)
num_classes = 9

In [11]:
# Gebruik een dummy input die overeenkomt met de input van je model
dummy_input = torch.randn(1, 3, 224, 224)

In [12]:
# Exporteer het model naar ONNX-formaat
torch.onnx.export(model, dummy_input, 'cropped_cnn.onnx', input_names=['input'], output_names=['output'], opset_version=11)

## Resultaten bekijken

In [14]:
def load_onnx_model(model_path):
    """
    Load an ONNX model and return the inference session.
    
    :param model_path: Path to the ONNX model file.
    :return: ONNX runtime inference session.
    """
    return ort.InferenceSession(model_path)

In [17]:
cropped_session = load_onnx_model("D:/Studie/Stage/Demo/model_structuur_vergelijken/Models/cropped image classification/cropped_cnn.onnx")
input_name = cropped_session.get_inputs()[0].name

In [ ]:
val_dataset = datasets.ImageFolder('data/imagenet_format_dataset/val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [19]:
val_dataset = datasets.ImageFolder('data/cropped_no_padding/val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [21]:
total_correct = 0
total_samples = 0

In [23]:
with torch.no_grad():
    for images, labels in val_loader:
        # Schakel PyTorch tensoren om naar numpy arrays
        images_np = images.numpy()

        # Doen voorspellingen
        outputs = cropped_session.run(None, {input_name: images_np})

        # Hier wordt verondersteld dat de uitvoer de logit/probabiliteitswaarden zijn
        predictions = np.argmax(outputs[0], axis=1)
        # print(predictions)

        # Bereken het aantal correcte voorspellingen
        total_correct += (predictions == labels.numpy()).sum()
        total_samples += labels.size(0)

In [24]:
# Print de nauwkeurigheid van het model
accuracy = total_correct / total_samples
print(f'Nauwkeurigheid: {accuracy * 100:.2f}%')

Nauwkeurigheid: 98.83%
